In [3]:
import openpyxl

def ensure_excelfile_exists(filePath) :
    try :
        # Make sure file exists.
        if os.path.exists( filePath ) is True :
            workbook = openpyxl.load_workbook( filePath )
        else :
            workbook = openpyxl.Workbook()
            workbook.save( filePath )
            # workbook = openpyxl.load_workbook( filePath )

        # # Make sure a sheet exists for this session.
        # if sheetName is not None and sheetName not in workbook.sheetnames :
        #     workbook.create_sheet( title = sheetName )

        # workbook.save( filePath )

    except :
        raise Exception( "Failed ensureing Excel file exists: {}".format(filePath) )

    return

In [ ]:
#================== create an empty excel sheet to edit bb_to_predict into.
import os

if not False:

    import pandas as pd
    Odds_cols = []
    for b in range(5):
        Odds_cols += ['HDA'+str(b)+'H', 'HDA'+str(b)+'D', 'HDA'+str(b)+'A']

    # input_columns = {'no': pd.Series(dtype='int'), 'year': pd.Series(dtype='int'), 'month': pd.Series(dtype='int'), 'day': pd.Series(dtype='int'),
    #     'Div': pd.Series(dtype='str'), 'HomeTeam': pd.Series(dtype='str'), 'AwayTeam': pd.Series(dtype='str')}
    input_columns = {'id': [], 'year': [], 'month': [], 'day': [], 'hour': [], 'minute': [], 'Div': [], 'HomeTeam': [], 'AwayTeam': []}
    input_columns = input_columns | {col: pd.Series(dtype='float') for col in Odds_cols}
    df = pd.DataFrame(input_columns)

    sheet = '6'
    filePath = os.path.join(".", "history", "test2.xlsx")
    ensure_excelfile_exists(filePath)
    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet)

In [ ]:
lists = [[1,2], [3,4]]
interleave = [val for tup in zip(*lists) for val in tup]
print(interleave)

[1, 3, 2, 4]


In [57]:
import pytz
from datetime import datetime

import YG_assistant

if not False:
    df = pd.read_excel(filePath, sheet_name=sheet)
    if 'Unnamed: 0' in list(df.columns): df.drop('Unnamed: 0', axis=1, inplace=True)
    # print(df)

    check_report = ""
    consistent = True
    # Check for consistency
    delta = list(set(list(input_columns.keys())) - set(df.columns))
    if len(delta) > 0: 
        consistent = False
        check_report += "Some columns are missing: {}\n".format(",".join(delta))
    if not (df.shape[0] > 0): 
        consistent = False
        check_report += "There are no data rows. \n"
    if df.isnull().any().any():
        consistent = False
        check_report += "There are some NaN values. \n"

    if set(['year', 'month', 'day', 'hour', 'minute']) <= set(df.columns):
        try:
            datetime_series = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']], utc=True)
            now = datetime.now(pytz.utc)
            delta_hours = [(dt - now).total_seconds()/3600 for dt in datetime_series]
            negatives = [id for id in range(len(delta_hours)) if delta_hours[id] < 0]
            if len(negatives) > 0:
                consistent = False
                check_report += "Some datetimes are past: indices = {}\n".format(",".join([str(id) for id in negatives]))
            aheads = [id for id in range(len(delta_hours)) if delta_hours[id] > 24]
            if max(delta_hours) > 24:
                consistent = False
                check_report += "Some datetimes are not within 24 hours: indices = {}\n".format(",".join([str(id) for id in aheads]))
        except:
            consistent = False
            check_report += "Some datetime columns have invalid data types or value range. \n"

    homes = list(df['HomeTeam']); aways = list(df['AwayTeam'])
    lists = [homes, aways]
    teamList = [val for tup in zip(*lists) for val in tup]
    # teams, candidates, success = YG_assistant.convert_to_token_with_candidates(tokenizer, teamList, candi_count=3)
    # if success in False:
    #     consistent = False
    #     check_report += "Mis-spelled teams and their candidates comes below: \n"
    #     for (spell, candis) in candidates.items():
    #         check_report += "{}: {}".format(spell, candis)

    DIVS = ['E0', 'E1', 'E2', 'E3']
    delta = list(set(df['Div']) - set(DIVS))
    if len(delta) > 0:
        consistent = False
        check_report += "Some Div values are wrong: {}\n".format(delta)

    import numpy as np

    NUMBER_BOOKIES = 5
    for col in Odds_cols:
        odds = np.array(df[col])
        condition = odds <= 1.0
        if condition.any():
            consistent = False
            check_report += "Some odds are too small: rows {} columns {}\n".format(list(np.where(condition)[0]), col)
        condition = odds > 60.0
        if (condition).any():
            consistent = False
            check_report += "Some odds are too large: rows {} columns {}\n".format(list(np.where(condition)[0]), col)

    for b in range(NUMBER_BOOKIES):
        cols = ['HDA'+str(b)+'H', 'HDA'+str(b)+'D', 'HDA'+str(b)+'A']
        odds = np.array(df[cols])
        condition = np.sum(1/odds, axis=-1) > 1.2
        if condition:
            consistent = False
            check_report += "Some odds group are too small: odds group {}\n".format(cols)
        condition = np.sum(1/odds, axis=-1) < 0.85
        if condition:
            consistent = False
            check_report += "Some odds group are too large: odds group {}\n".format(cols)

    print(consistent)
    print(check_report)

    # if consistent:  # convert to bbab
    start = 1000000 * 2
    df['id'] = list(range(start, start + df.shape[0]))
    print(df)

    

False
Some columns are missing: no
Some Div values are wrong: [23]
Some odds are too small: rows [0] columns HDA3H
Some odds are too small: rows [0] columns HDA3D
Some odds group are too large: odds group ['HDA1H', 'HDA1D', 'HDA1A']
Some odds group are too small: odds group ['HDA2H', 'HDA2D', 'HDA2A']
Some odds group are too small: odds group ['HDA3H', 'HDA3D', 'HDA3A']
Some odds group are too small: odds group ['HDA4H', 'HDA4D', 'HDA4A']

        id  year  month  day  hour  minute  Div  HomeTeam  AwayTeam  HDA0H  \
0  2000000  2025      4   12    12      30   23         1         1    2.6   

   ...  HDA1A  HDA2H  HDA2D  HDA2A  HDA3H  HDA3D  HDA3A  HDA4H  HDA4D  HDA4A  
0  ...      9      2      2      2      1      1      2      2      2      2  

[1 rows x 24 columns]


In [82]:
import pytz
from datetime import datetime
now = datetime.utcnow().replace(tzinfo=pytz.utc)
print(now.date())

print(datetime.now(pytz.utc).date())

2025-04-11
2025-04-11


In [ ]:
import pandas as pd

data = { 'col1': [1,1,3], 'col2': [1,1,5], 'col3': [1,2,3] }
df = pd.DataFrame(data)
print(df)
df = df.drop_duplicates(subset=['col1', 'col2'])
print(df)

   col1  col2  col3
0     1     1     1
1     2     2     2
2     3     5     3
   col1  col2  col3
0     1     1     1
1     2     2     2
2     3     5     3


In [4]:

def generator(aa):
    for a in aa:
        if aa.index(a) < 4:
            yield a
        else:
            aa = [a + 5 for a in aa]
            yield generator(aa)

bb = generator([1,2,3,4,5])

for b in bb:
    print(b)

1
2
3
4
<generator object generator at 0x00000205F78CC040>


In [2]:
d = { 1: (11, 12), 2: (22, 34) }
f = { id: value for (id, value) in d.items() if id > 1}
print(f)

{2: (22, 34)}


In [7]:
import tensorflow as tf
a = tf.constant([[1,2,3,4,5], [6,7,8,9,0]])
tf.gather(a, [0, 2,3], axis=-1)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 3, 4],
       [6, 8, 9]])>

In [ ]:
import tensorflow as tf

len = tf.constant([1,2,3], dtype=tf.int32)

total_mask = []
for s in len:
    mask = tf.concat([tf.ones(s, dtype=tf.int32), tf.zeros(5-s, dtype=tf.int32)], axis=-1)
    mask = mask[:, tf.newaxis] & mask[tf.newaxis, :]
    total_mask.append(mask)
    print(mask)
total_mask = tf.stack(total_mask, axis=0)
print(total_mask.shape)
print(total_mask)


tf.Tensor(
[[1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
tf.Tensor(
[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
tf.Tensor(
[[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
(3, 5, 5)
tf.Tensor(
[[[1 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 0 0 0]
  [1 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 0 0]
  [1 1 1 0 0]
  [1 1 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]], shape=(3, 5, 5), dtype=int32)


In [39]:
import pandas as pd
import numpy as np

data = { 'col1': [1, 2, 3, 4, np.nan], 'col2': [1, np.nan, 3, 4, 5], 'col3': [1, 2, np.nan, 4, np.nan] }
df = pd.DataFrame(data)

df2 = df.loc[:, ['col1', 'col3']]
print(df2)
array = df2['col1'].dropna()
print(array)

df2.loc[:, 'col1'] = -3

for index, row in df.iterrows():
    # if pd.isnull(row['col1']) or pd.isnull(row['col2']):
    #     print(row['col1'], row['col2'])
    if pd.isnull( row['col1'] ):
        row['col1'] = -5

print(df)
# df.loc[df.isnull().any(axis=1)]

   col1  col3
0   1.0   1.0
1   2.0   2.0
2   3.0   NaN
3   4.0   4.0
4   NaN   NaN
0    1.0
1    2.0
2    3.0
3    4.0
Name: col1, dtype: float64
   col1  col2  col3
0   1.0   1.0   1.0
1   2.0   NaN   2.0
2   3.0   3.0   NaN
3   4.0   4.0   4.0
4  -5.0   5.0   NaN


In [4]:
import os
COUNTRY = 'England'
THEME = 'test_theme'
CountryThemeFolderPath = "./data/football-data-co-uk/" + COUNTRY + '/' + THEME
os.makedirs(CountryThemeFolderPath, exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_id_map'), exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_dataaset'), exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_checkpoint'), exist_ok=True)

In [5]:
import http.client

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}
conn.request("GET", "/v3/football/fixtures/head-to-head/{team_id_1}/{team_id2}?api_token=YOUR_TOKEN", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

gaierror: [Errno 11001] getaddrinfo failed

In [ ]:
print(None)

In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# !pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
# !pip install protobuf~=3.20.3
# !pip install -q tensorflow_datasets
# !pip install -q -U tensorflow-text tensorflow

In [ ]:
#  no name 'resource'?
#  pip3 uninstall tensorflow_datasets; pip3 install tensorflow_datasets==4.9.2

In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import os
import data_helpers
import data_generator
from config import config

In [ ]:
df = data_helpers.read_large_excel(config['data'], "contest_total_data")
df = data_helpers.improve_contest_data(df)

In [ ]:
teams = list(set(list(df['teams_home_team_id']) + list(df['teams_away_team_id'])))
teams_string = [str(team) for team in teams]
teams_text = " ".join(teams_string)

corpus_file = "./data/tokenizers/team_ids_text.txt"
f = open(corpus_file, "w", encoding="utf-8")
f.write(teams_text)
f.close()

corpus_files = [corpus_file]
unknown_token = config['unknown_token']
special_tokens = [unknown_token] + ["[HOME]", "[AWAY]"]
vocab_size = len(teams_string) + len(special_tokens)

tokenizer_team = data_generator.createSimpleTokenizer(corpus_files, vocab_size, unknown_token, special_tokens)

In [ ]:
tokenizer_team.get_vocab_size()

In [ ]:
encoding = tokenizer_team.encode("1 2 3 what? 333 -5 4)e_t grw0 1234 [HOME] [AWAY]")
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

tokenizer_team.decode(encoding.ids)

In [ ]:
tokenizer_team.decode([3, 2046, 2921, 0, 3105, 0, 0, 0, 804, 1, 2])

In [ ]:
import re

countries_string = list(set(list(df['league_country']) + list(df['home_team_country'])+ list(df['away_team_country'])))
# print(countries_string.index('ViÃ° MargÃ¡ir'))
countries_string = [re.sub(r"\s", "_", item) for item in countries_string]    # replace spaces with a '_'
# print(countries_string.index('ViÃ°_MargÃ¡ir'))
# teams_string = [str(team) for team in teams]
countries_text = " ".join(countries_string)

corpus_file = "./data/tokenizers/country_ids_text.txt"
f = open(corpus_file, "w", encoding="utf-8")
f.write(countries_text)
f.close()

corpus_files = [corpus_file]
unknown_token = config['unknown_token']
special_tokens = [unknown_token] + ["[LEAGUE_COUNTRY]", "[HOME_COUNTRY]", "[AWAY_COUNTRY]"]
vocab_size = len(countries_string) + len(special_tokens)

tokenizer_country = data_generator.createSimpleTokenizer(corpus_files, vocab_size, unknown_token, special_tokens)

In [ ]:
countries = ['Argentina', 'Italy', 'WHAT', 'France', 'USA', 'Canada', '[UNK]', 'Unknown', '[AWAY_COUNTRY]', 'Chinese Taipei', 'ViÃ° MargÃ¡ir']
countries = [re.sub(r"\s", "_", item) for item in countries]
print(countries)
countries = " ".join(countries)
encoding = tokenizer_country.encode(countries)
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

tokenizer_country.decode(encoding.ids)

In [ ]:
filename = 'ids-5.0-0.01-15.0-0.9-0.8-10-0.000005-26.64-32601-15'
(a_conductance_search, b_conductance_search, conductance365, 
        conductanceMedTeam, ids_max, minConductanceStep, hours, nDataPoints, maxLen) \
= data_helpers.filename_to_dataParams(filename)
path = os.path.join('./data', filename + '.json')
fixture_id_to_ids = data_helpers.LoadJsonData(path)

In [ ]:
lengths = [len(ids) for ids in fixture_id_to_ids.values()]
plt.hist(lengths, np.linspace(0, 200, 101))
plt.ylim(plt.ylim())
maxLen = max(lengths)
plt.plot([maxLen, maxLen], plt.ylim())
plt.title(f'Max length of ids: {maxLen}')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
for elem in dataset:
    print(elem)

In [ ]:
ds

In [ ]:
pair_cols = ['teams_home_team_id', 'teams_away_team_id']
dummy_pair = "[HOME] [AWAY]"
country_cols = ['league_country', 'home_team_country', 'away_team_country']
dummy_countries = "[LEAGUE_COUNTRY] [HOME_COUNTRY] [AWAY_COUNTRY]"
odds_cols = ['winning_odds_home', 'winning_odds_draw', 'winning_odds_away']
dummy_wining_percent = (80, 10, 10)         
dummy_odds = [
    data_helpers.get_odds(dummy_wining_percent[0]/100, config['bookie_profit_percent']),
    data_helpers.get_odds(dummy_wining_percent[1]/100, config['bookie_profit_percent']),
    data_helpers.get_odds(dummy_wining_percent[2]/100, config['bookie_profit_percent']),
]
outcome_cols = ['outcome']
dummy_outcome = [1, 0, 0]


In [ ]:
def get_BB(df, game_id):
    try:
        if game_id >= 0:
            pair = list(df.loc[df['fixture_id'] == game_id, pair_cols].iloc[0])
            pair_str = [str(team) for team in pair]
            pair_text = " ".join(pair_str)
            pair_tokens = tokenizer_team.encode(pair_text).ids
            print(pair_text, pair_tokens)
            countries = list(df.loc[df['fixture_id'] == game_id, country_cols].iloc[0])
            countries = [re.sub(r"\s", "_", item) for item in countries]
            countries_text = " ".join(countries)
            countries_tokens = tokenizer_country.encode(countries_text).ids
            print(countries_text, countries_tokens)
            odds = list(df.loc[df['fixture_id'] == game_id, odds_cols].iloc[0])
            print(odds)
        elif game_id == -1:
            pair_text = dummy_pair
            pair_tokens = tokenizer_team.encode(pair_text).ids
            print(pair_text, pair_tokens)
            countries_text = dummy_countries
            countries_tokens = tokenizer_country.encode(countries_text).ids
            print(countries_text, countries_tokens)      
            odds = dummy_odds
            print(odds)

        BB = pair_tokens + countries_tokens + odds
        BB = np.array(BB)
        print(BB)
        return BB
    except:
        raise Exception("Failed to get_BB. game_id: {}".format(game_id))

def get_AB(df, game_id):
    try:
        if game_id >= 0:
            outcome = list(df.loc[df['fixture_id'] == game_id, outcome_cols].iloc[0])
            outcome = (1, 0, 0) if outcome == 0 else (0, 1, 0) if outcome == 1 else (0, 0, 1)
        elif game_id == -1:   # Hyperthetic game
            print("id=-1")
            outcome = dummy_outcome     # [HOME] team always wins.
            print(outcome)
        
        AB = outcome
        AB = np.array(AB, dtype=np.float32)
        print(AB)
        return AB
    except:
        raise Exception("Failed to get_AB. game_id: {}".format(game_id))

In [ ]:
def generator():    
    for baseId, ids in fixture_id_to_ids.items():
        print(baseId, ids)
        sequence = []
        if len(ids) > 0:
            for id in ids:
                BB = get_BB(df, id)
                AB = get_AB(df, id)
                if BB.shape[0] + AB.shape[0] > 11:
                    print("shape", baseId, ids)
                sequence.append(np.concatenate((BB, AB)))
                print("1", sequence[-1].shape)
        else:
            BB = get_BB(df, -1)
            AB = get_AB(df, -1)
            sequence.append(np.concatenate((BB, AB)))
            print("1", sequence[-1].shape)

        sequence = np.stack(sequence, axis=0)
        print("shape", sequence.shape)
        BB = get_BB(df, int(baseId))
        AB = get_AB(df, int(baseId))
        if BB.shape[0] + AB.shape[0] > 11:
            print("shape", baseId, ids)
        baseOutput = np.concatenate((BB, AB))

        totalOutput = (baseOutput, sequence)
        print("total: ", totalOutput)

        yield totalOutput


In [ ]:
ds = tf.data.Dataset.from_generator(
    generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape((11,)), tf.TensorShape((None, 11))),
    args=()
)

In [ ]:
for item in ds:
    print(item)
    # break

In [ ]:
tf.data.experimental.save(ds, "./data/dataset")

In [ ]:
ds2 = tf.data.experimental.load("./data/dataset")

In [ ]:
for item in ds2:
    print(item)

In [ ]:
generator()